In [ ]:
import numpy as np
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras.models import load_model
import cv2

In [ ]:
def generate_dir(directory):
    try:
        os.stat(directory)
    except:
        os.mkdir(directory)

def get_output_layer(model, layer_name):
    # get the symbolic outputs of each "key" layer (we gave them unique names).
    layer_dict = dict([(layer.name, layer) for layer in model.layers])
    layer = layer_dict[layer_name]
    return layer

def visualize_class_activation_map_v3(model, img_path, output_path, group):
    
    """
    change group value for different groups
    
    """
    
    model = model
    original_img = cv2.imread(img_path, 1)
    # print(img_path)
    width, height, _ = original_img.shape
    
    img = cv2.resize(original_img, dsize=(224, 224), interpolation=cv2.INTER_LANCZOS4) #
    img = img/255.  #
    img = img[np.newaxis,::,::,::] #
    
    class_weights = model.layers[-1].get_weights()[0]
    final_conv_layer = get_output_layer(model, "mixed10") # 
    get_output = K.function([model.layers[0].input], [final_conv_layer.output, model.layers[-1].output])
    [conv_outputs, predictions] = get_output([img])
    conv_outputs = conv_outputs[0, :, :, :]

    #Create the class activation map.
    cam = np.zeros(dtype = np.float32, shape = conv_outputs.shape[0:2]) #
    for i, w in enumerate(class_weights[:, group]): #
        cam += w * conv_outputs[:, :, i]
    print("predictions", predictions)
    cam /= np.max(cam)
    cam = cv2.resize(cam, (height, width))
    heatmap = cv2.applyColorMap(np.uint8(255*cam), cv2.COLORMAP_JET)
    heatmap[np.where(cam < 0.2)] = 0 # 0.002
    img = heatmap*0.5 + original_img# 0.5
    cv2.imwrite(output_path, img)


In [ ]:
# load google inception v3 model
model_path = "...h5"
model = load_model(model_path)

In [ ]:
import os, glob
import pandas as pd


dcm_dir = "..." # file dir of the echo images 
file_name = [x for x in glob.glob(dcm_dir + "*.png")]
file_name = pd.DataFrame(data=file_name, columns = ["dir_"])
file_name["patients"] = ""

for idx, y in enumerate(file_name.dir_):
    y1 = y.split("\\")[-1]
    length = len(y1.split("_")[-1])
    y2 = y1[:-(length+1)]
    file_name.loc[idx,"patients"] = y2

In [ ]:
for idx, x in enumerate(input_dir):
    
    group = 1
    output_path = os.path.join(dir_output_folder)
    generate_dir(directory=output_path)
    
    
    dataframe = file_name[file_name["patients"]==x]
    
    if dataframe.empty:
        print(x)
        print("\t")
    else:
        for idx2, y in enumerate(dataframe["dir_"]):
            img_path = y
            directory = output_path + x
            generate_dir(directory=directory)

            fil = img_path.split("\\")[-1]
            output_path2 = directory + "\\" + fil
            visualize_class_activation_map_v3(model, img_path, output_path2, group) 